# Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

## Import all necessary packages

In [1]:
import pandas as pd
import numpy as np

## Modifying data

### Drop useless stuff, rename column names

In [2]:
# read Excel with all data and drop some useless data
energy_df = pd.read_excel("Energy Indicators.xls", skiprows=16, skipfooter=38).drop(['Unnamed: 0', 'Unnamed: 1'], axis=1).drop(0)

# rename [Unnamed: 2] -> [Country], [Renewable Electricity Production] -> [% Renewable]
energy_df = energy_df.rename(columns = {'Unnamed: 2': 'Country', 'Renewable Electricity Production': '% Renewable'})

energy_df.head(5)

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,321,10,78.66928
2,Albania,102,35,100
3,Algeria,1959,51,0.55101
4,American Samoa,...,...,0.641026
5,Andorra,9,121,88.69565


### Replace missing data (e.g. data with `...`) with `np.NaN` values.

In [3]:
cols = list(energy_df.columns)[1:]

# First method
energy_df[cols] = energy_df[cols].apply(lambda x: [item if isinstance(item, (int, float)) else np.NaN for item in x.values])

# Second method
# mask = df[cols].applymap(lambda x: isinstance(x, (int, float)))
# df[cols] = df[cols].where(mask)

print(energy_df.head(5))

energy_df.dtypes

          Country  Energy Supply  Energy Supply per capita  % Renewable
1     Afghanistan          321.0                      10.0    78.669280
2         Albania          102.0                      35.0   100.000000
3         Algeria         1959.0                      51.0     0.551010
4  American Samoa            NaN                       NaN     0.641026
5         Andorra            9.0                     121.0    88.695650


Country                      object
Energy Supply               float64
Energy Supply per capita    float64
% Renewable                 float64
dtype: object

### Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).

In [4]:
# Converting
energy_df['Energy Supply'] = energy_df['Energy Supply']*1_000_000

energy_df.head(5)

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,3.210000e+08,10.0,78.669280
2,Albania,1.020000e+08,35.0,100.000000
3,Algeria,1.959000e+09,51.0,0.551010
4,American Samoa,NaN,NaN,0.641026
5,Andorra,9.000000e+06,121.0,88.695650


### Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`

In [5]:
energy_df = energy_df.replace({ 'Country': {'Republic of Korea': 'South Korea',
                         'United States of America\d+': 'United States',
                         'United Kingdom of Great Britain and Northern Ireland\d+': 'United Kingdom',
                         'China, Hong Kong Special Administrative Region\d+': 'Hong Kong'}}, regex=True)

# check if replace was successful
[energy_df.loc[energy_df['Country'] == country] for country in ['Hong Kong', 'South Korea', 'United States', 'United Kingdom']]


    

[      Country  Energy Supply  Energy Supply per capita  % Renewable
 44  Hong Kong    585000000.0                      82.0          0.0,
          Country  Energy Supply  Energy Supply per capita  % Renewable
 165  South Korea   1.100700e+10                     221.0     2.279353,
            Country  Energy Supply  Energy Supply per capita  % Renewable
 217  United States   9.083800e+10                     286.0     11.57098,
             Country  Energy Supply  Energy Supply per capita  % Renewable
 215  United Kingdom   7.920000e+09                     124.0     10.60047]

### There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.

In [6]:
# replace:
# Switzerland17 -> Switzerland
# Bolivia (Plurinational State of) -> Bolivia
energy_df = energy_df.replace({ 'Country': { r'([A-z]+)\d+': r'\1', r'(\w+) \(.*\)': r'\1' } }, regex=True)

[energy_df.loc[energy_df['Country'] == country] for country in ['Switzerland', 'Bolivia']]

[         Country  Energy Supply  Energy Supply per capita  % Renewable
 198  Switzerland   1.113000e+09                     136.0     57.74548,
     Country  Energy Supply  Energy Supply per capita  % Renewable
 25  Bolivia    336000000.0                      32.0     31.47712]

### Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD).

It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"

Make sure to skip the header, and rename the following list of countries:
- `Korea, Rep.`: `South Korea`,
- `Iran, Islamic Rep.`: `Iran`,
- `Hong Kong SAR, China`: `Hong Kong`

In [7]:
GDP_df = pd.read_excel('world_bank.xls', skiprows=3)

GDP_df.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.549721e+09,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09,2.983637e+09,3.092430e+09,3.202189e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,1.929193e+10,1.970186e+10,2.147035e+10,2.570500e+10,2.350165e+10,2.678117e+10,...,8.962561e+11,9.131974e+11,9.276555e+11,9.563187e+11,8.930997e+11,8.547519e+11,9.622690e+11,9.840320e+11,9.778092e+11,8.984741e+11


In [8]:
GDP_df = GDP_df.rename(columns={ 'Country Name': 'Country' })

GDP_df = GDP_df.replace({ 'Country': { 'Korea, Rep.': 'South Korea', 'Iran, Islamic Rep.': 'Iran', 'Hong Kong SAR, China': 'Hong Kong' } })

[GDP_df.loc[GDP_df['Country'] == country] for country in ['South Korea', 'Iran', 'Hong Kong']]

[         Country Country Code     Indicator Name  Indicator Code  \
 126  South Korea          KOR  GDP (current US$)  NY.GDP.MKTP.CD   
 
              1960          1961          1962          1963          1964  \
 126  3.958191e+09  2.417558e+09  2.814319e+09  3.988785e+09  3.458939e+09   
 
              1965  ...          2011          2012          2013  \
 126  3.120833e+09  ...  1.253223e+12  1.278428e+12  1.370795e+12   
 
              2014          2015          2016          2017          2018  \
 126  1.484318e+12  1.465773e+12  1.500112e+12  1.623901e+12  1.724846e+12   
 
              2019          2020  
 126  1.646739e+12  1.630525e+12  
 
 [1 rows x 65 columns],
     Country Country Code     Indicator Name  Indicator Code          1960  \
 112    Iran          IRN  GDP (current US$)  NY.GDP.MKTP.CD  4.199134e+09   
 
              1961          1962          1963          1964          1965  \
 112  4.426949e+09  4.693566e+09  4.928628e+09  5.379846e+09  6.197320e+

## Read the last dataset
- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

In [9]:
ScimEn_df = pd.read_excel('scimagojr country rank 1996-2020.xlsx')

ScimEn_df.head(5)

,Rank,Country,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,Asiatic Region,273437,272374,2336764,1615239,8.55,245
1,2,United States,Northern America,175891,172431,2230544,724472,12.68,363
2,3,India,Asiatic Region,55082,53775,463165,162944,8.41,181
3,4,Japan,Asiatic Region,50523,50065,488062,119930,9.66,193
4,5,United Kingdom,Western Europe,43389,42284,615670,111290,14.19,226


#### Merge all datasets

In [10]:

merged_df = GDP_df.merge(energy_df, on='Country').merge(ScimEn_df, on='Country').set_index('Country')

merged_df = merged_df.filter(items=['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'])

merged_df = merged_df.sort_values(by=['Rank'])

merged_df.head(5)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,273437,272374,2336764,1615239,8.55,245,1.271910e+11,93.0,19.75491,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,175891,172431,2230544,724472,12.68,363,9.083800e+10,286.0,11.57098,1.381461e+13,1.445186e+13,1.471284e+13,1.444893e+13,1.499205e+13,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.823830e+13
India,3,55082,53775,463165,162944,8.41,181,3.319500e+10,26.0,14.96908,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,50523,50065,488062,119930,9.66,193,1.898400e+10,149.0,10.23282,4.530377e+12,4.515265e+12,5.037908e+12,5.231383e+12,5.700098e+12,6.157460e+12,6.203213e+12,5.155717e+12,4.850414e+12,4.389476e+12
United Kingdom,5,43389,42284,615670,111290,14.19,226,7.920000e+09,124.0,10.60047,2.706015e+12,3.093407e+12,2.921432e+12,2.412390e+12,2.481580e+12,2.659882e+12,2.704017e+12,2.783251e+12,3.065521e+12,2.932785e+12


### Create function `answer_one` that returns top-15 countries by Rank

In [23]:
def answer_one(df=merged_df):
    '''Returns top-15 countries by rank
    '''
    return df.loc[df['Rank'] < 16]

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,273437,272374,2336764,1615239,8.55,245,1.271910e+11,93.0,19.754910,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,175891,172431,2230544,724472,12.68,363,9.083800e+10,286.0,11.570980,1.381461e+13,1.445186e+13,1.471284e+13,1.444893e+13,1.499205e+13,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.823830e+13
India,3,55082,53775,463165,162944,8.41,181,3.319500e+10,26.0,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,50523,50065,488062,119930,9.66,193,1.898400e+10,149.0,10.232820,4.530377e+12,4.515265e+12,5.037908e+12,5.231383e+12,5.700098e+12,6.157460e+12,6.203213e+12,5.155717e+12,4.850414e+12,4.389476e+12
United Kingdom,5,43389,42284,615670,111290,14.19,226,7.920000e+09,124.0,10.600470,2.706015e+12,3.093407e+12,2.921432e+12,2.412390e+12,2.481580e+12,2.659882e+12,2.704017e+12,2.783251e+12,3.065521e+12,2.932785e+12
Germany,6,38739,38013,433148,95145,11.18,196,1.326100e+10,165.0,17.901530,2.992197e+12,3.421229e+12,3.730028e+12,3.397791e+12,3.396354e+12,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.356236e+12
Russian Federation,7,36735,36560,115938,54993,3.16,90,3.070900e+10,214.0,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,33472,32863,568080,100953,16.97,227,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,27983,26940,352993,87828,12.61,166,6.530000e+09,109.0,33.667230,1.947920e+12,2.210293e+12,2.398857e+12,2.191242e+12,2.134018e+12,2.291991e+12,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12


## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [22]:
def answer_two():
    '''Calculate average GDP for top-15 countries'''
    # Get top-15 values
    Top15 = answer_one()

    # Get only [Year] columns
    avgGDP = Top15.filter(regex='\d+')

    # Calculate the average GDP for each country and sort by descending
    avgGDP = avgGDP.mean(axis=1).sort_values(ascending=False)
    
    return avgGDP

answer_two()

Country
United States         1.567102e+13
China                 6.927702e+12
Japan                 5.177131e+12
Germany               3.518225e+12
United Kingdom        2.776028e+12
France                2.687371e+12
Italy                 2.139775e+12
Brazil                1.988885e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.398587e+12
South Korea           1.221372e+12
Australia             1.206526e+12
Iran                  4.389286e+11
dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [44]:
def answer_three():
    '''Calculate how GDP was changed over the 10 years for 6th largst average GDP
    '''
    # get top-15 values
    Top15 = answer_one()

    # get 6th largest average GDP
    sixth_country_gdp = Top15.filter(items=['2006', '2015'], axis=1).loc['France']

    return sixth_country_gdp[1] - sixth_country_gdp[0]

answer_three()

119614244263.3833

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [80]:
def answer_four():
    '''Calculate the ratio of Self-citations to all Citations.
       Find the highest ratio and which country is it.
    '''
    # Get top-15 values
    Top15 = answer_one()

    # Get neeeded columns
    df_citations = Top15.filter(items=['Citations', 'Self-citations'])

    # Calculating [Citations Ratio]
    df_citations['Citations Ratio'] = df_citations['Self-citations']/df_citations['Citations']

    # Find the highest value
    max_cit_ratio = df_citations['Citations Ratio'].max()
    
    return (max_cit_ratio, df_citations.index[df_citations['Citations Ratio'] == max_cit_ratio].tolist()[0])

answer_four()

(0.6912289816173135, 'China')

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [90]:
\
def answer_five():
    '''Calculate population for top-15 countries using energy
    '''
    # get top-15 values
    Top15 = answer_one()

    # get needed values
    energy = Top15.filter(items=['Energy Supply', 'Energy Supply per capita'])

    # calculate population
    energy['Population'] = energy['Energy Supply']/energy['Energy Supply per capita']

    # sort values in [Population] column by descending
    energy = energy.sort_values(by='Population', ascending=False)
    
    return energy.index[energy['Population'] == energy['Population'][2]].tolist()[0]

answer_five()

'United States'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [102]:
def answer_six():
    '''Calculate correlation between citable doc per capita and energy supply per capita
    '''
    # get top 15 values
    Top15 = answer_one()

    # get needed info
    taskSix_df = Top15.filter(items=['Citable documents', 'Energy Supply', 'Energy Supply per capita'])

    # calculate population
    taskSix_df['Population'] = taskSix_df['Energy Supply']/taskSix_df['Energy Supply per capita']

    # calculate citable documents per capita
    taskSix_df['Citable documents per capita'] = taskSix_df['Citable documents']/taskSix_df['Population']
    
    return taskSix_df['Citable documents per capita'].corr(taskSix_df['Energy Supply per capita'])

answer_six()

0.7434709127726777

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [170]:
def answer_seven():
    # get top-15 values
    Top15 = answer_one()

    # get needed info
    taskSeven_df = Top15.filter(items=['Energy Supply', 'Energy Supply per capita'])

    # calculate population
    taskSeven_df['Population'] = taskSeven_df['Energy Supply']/taskSeven_df['Energy Supply per capita']

    # create dict {[country]: [continent]}
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

    # add [Continent] column
    taskSeven_df['Continent'] = [ContinentDict[x] for x in taskSeven_df.index.tolist()]

    # get continents list
    continent_indexes = taskSeven_df.groupby('Continent').count().index.tolist()

    # find how much countries we have per continent
    countries_per_continent = taskSeven_df['Continent'].value_counts().sort_index().tolist()
    # print(taskSeven_df['Continent'].value_counts().sort_index().tolist())

    # sum population
    sum_population = taskSeven_df.groupby(['Continent']).sum()['Population'].tolist()
    # print(taskSeven_df.groupby(['Continent']).sum()['Population'].tolist())

    # mean population
    mean_population = taskSeven_df.groupby(['Continent']).mean()['Population'].tolist()
    # print(taskSeven_df.groupby(['Continent']).mean()['Population'].tolist())

    # std population
    std_population = taskSeven_df.groupby(['Continent']).std()['Population'].tolist()
    # print(taskSeven_df.groupby(['Continent']).std()['Population'])

    # create result df
    result_df = pd.DataFrame({ 'size': countries_per_continent, 'sum': sum_population, 'mean': mean_population, 'std': std_population }, index=continent_indexes, columns=['size', 'sum', 'mean', 'std'])

    return result_df

answer_seven()


Continent
Asia             6.790979e+08
Australia                 NaN
Europe           3.464767e+07
North America    1.996696e+08
South America             NaN
Name: Population, dtype: float64


NameError: name 'std_population' is not defined